In [1]:
# disable Jedi for better autocomplete
%config Completer.use_jedi = False

# you can make it permanent by adding the following line to your Jupyter config file
# ~/.jupyter/jupyter_notebook_config.py
# c.Completer.use_jedi = False

In [2]:
#Importing openCV
import cv2

#Displaying image

image = cv2.imread('test_image.jpg')
cv2.imshow('input_image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Converting the image to grayscale

In [3]:
import cv2
import numpy as np

In [4]:
image = cv2.imread('test_image.jpg')

In [5]:
lanelines_image = np.copy(image)

In [6]:
gray_conversion= cv2.cvtColor(lanelines_image, cv2.COLOR_BGR2GRAY)

In [7]:
#Displaying grayscale image

cv2.imshow('input_image', gray_conversion)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Smoothing the image

In [8]:
import cv2
import numpy as np

In [9]:
image = cv2.imread('test_image.jpg')
lanelines_image = np.copy(image)

In [10]:
gray_conversion= cv2.cvtColor(lanelines_image, cv2.COLOR_BGR2GRAY)

In [11]:
# Uses a Gaussian filter to smooth the image.
# (5,5) is the kernel size (must be odd numbers like 3×3, 5×5, etc.).
# The 0 at the end is the standard deviation (calculated automatically if set to 0).
# Why Apply Blurring? 
# ✔ Reduces noise and unwanted small details.
# ✔ Helps edge detection algorithms (like Canny) to focus on important edges.
blur_conversion = cv2.GaussianBlur(gray_conversion, (5, 5), 0)

In [12]:
cv2.imshow('input_image', blur_conversion)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Canny edge detection

An edge is a region in an image where there is a sharp change in intensity or a sharp change in color between adjacent pixels in an image. This change over a series of pixels is known as the gradient.
The canny function computes the gradient in all directions of a blurred image and will trace the strongest gradient as a series of pixels.

How Canny Edge Detection Works:
- Noise Reduction: Uses a Gaussian filter to smooth the image.
- Gradient Calculation: Computes intensity gradients using Sobel operators.
- Non-Maximum Suppression: Thin out edges to remove unwanted pixels.
- Hysteresis Thresholding:
    - Edges above threshold2 (155) are strong edges (kept).
    - Edges below threshold1 (50) are discarded.
    - Edges between 50-155 are kept only if connected to a strong edge.


In [13]:
import cv2
import numpy as np

In [14]:
image = cv2.imread('test_image.jpg')

In [15]:
lanelines_image = np.copy(image)

In [16]:
gray_conversion= cv2.cvtColor(lanelines_image, cv2.COLOR_BGR2GRAY)

In [17]:
blur_conversion = cv2.GaussianBlur(gray_conversion, (5, 5), 0)

In [18]:
canny_conversion = cv2.Canny(blur_conversion, 50, 155) # threshold1 - 50, threshold2 - 155; If you don’t set apertureSize, OpenCV defaults to 3, meaning it uses a 3×3 Sobel kernel.

In [19]:
# we can see that the strongest gradients are represented by the color white
cv2.imshow('input_image', canny_conversion)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Masking the region of interest

In [20]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [21]:
def canny_edge(image):
          gray_conversion= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          blur_conversion = cv2.GaussianBlur(gray_conversion, (5, 5), 0)
          canny_conversion = cv2.Canny(blur_conversion, 50, 150)
          return canny_conversion


In [22]:
def reg_of_interest(image):
        # The height helps define the bottom part of the region.
        Image_height = image.shape[0]
        # Defines a triangular polygon with 3 points:
        # (200, Image_height) → Bottom-left corner.
        # (1100, Image_height) → Bottom-right corner.
        # (550, 250) → Apex (top-middle)
        # Why this shape?
        # ✅ It focuses on the road ahead where lane lines appear.
        # ✅ Excludes irrelevant areas like the sky, buildings, and nearby cars.
        # polygons = np.array([[(200, Image_height), (1100, Image_height), (550, 250)]])
        
        # 4-dimensional polygon
        polygons = np.array([[(0, int(Image_height)), (1100, Image_height), (550, 250), (0, int(Image_height*3/4))]])
        # Creates an empty black image (same size as input).
        # Used for masking.
        image_mask = np.zeros_like(image)
        # Fills the defined polygon with white (255, 255, 255) on the black mask.
        # This allows processing only inside the region.
        cv2.fillPoly(image_mask, polygons, (255, 255, 255))
        return image_mask

In [23]:
image = cv2.imread('test_image.jpg')
print(image.shape)
lanelines_image = np.copy(image)
canny_conversion = canny_edge(lanelines_image)

(704, 1279, 3)


In [24]:
cv2.imshow('result', reg_of_interest(canny_conversion))
cv2.waitKey(0)
cv2.destroyAllWindows()

### Applying bitwise_and

In [25]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [26]:
def canny_edge(image):
         gray_conversion= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
         blur_conversion = cv2.GaussianBlur(gray_conversion, (5,5),0)
         canny_conversion = cv2.Canny(blur_conversion, 50,150)
         return canny_conversion

In [27]:
def reg_of_interest(image):
         image_height = image.shape[0]
         # polygons = np.array([[(200, image_height), (1100, image_height), (551, 250)]])
         
         # 4-dimensional polygon
         polygons = np.array([[(0, int(image_height)), (1100, image_height), (550, 250), (0, int(image_height*3/4))]])
         image_mask = np.zeros_like(image)
         cv2.fillPoly(image_mask, polygons, (255, 255, 255)) # The mask now has a white polygon on a black background.
         
         # bitwise_and multiplies all the bits in the black region of the image by 0 (all these parts becomes black in the original image) and the white region by 1111 (keeping the pixels in the original image in that area).
         masking_image = cv2.bitwise_and(image, image_mask) # keeps only the polygon area from the original (e.g. cannied) image
         
         return masking_image

In [28]:
image = cv2.imread('test_image.jpg')

In [29]:
lanelines_image = np.copy(image)

In [30]:
canny_conversion = canny_edge(lanelines_image)

In [31]:
cropped_image = reg_of_interest(canny_conversion)

In [32]:
cv2.imshow('result', cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Applying the Hough transform

In [33]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [34]:
def canny_egde(image):
         gray_conversion= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
         blur_conversion = cv2.GaussianBlur(gray_conversion, (5,5),0)
         canny_conversion = cv2.Canny(blur_conversion, 50,150)
         return canny_conversion

In [35]:
def reg_of_interest(image):
         image_height = image.shape[0]
         # polygons = np.array([[(200, image_height), (1100, image_height), (551, 250)]])
            
         # 4-dimensional polygon
         polygons = np.array([[(0, int(image_height)), (1100, image_height), (550, 250), (0, int(image_height*3/4))]])
         image_mask = np.zeros_like(image)
         cv2.fillPoly(image_mask, polygons, (255, 255, 255))
         masking_image = cv2.bitwise_and(image, image_mask)
         return masking_image

In [36]:
def show_lines(image, lines):
            lines_image = np.zeros_like(image)
            if lines is not None:
                for line in lines:
                    # line is a NumPy array of shape (1, 4), so reshape(4) converts it into four separate values:
                    # (X1, Y1) → Start point of the line.
                    # (X2, Y2) → End point of the line.
                    X1, Y1, X2, Y2 = line.reshape(4)
                    # Draws a blue line ((255,0,0)) between (X1, Y1) and (X2, Y2). Line thickness = 10 pixels.
                    # Color Format: OpenCV uses BGR, not RGB, so:
                    # (255, 0, 0) = Blue
                    # (0, 255, 0) = Green
                    # (0, 0, 255) = Red
                    cv2.line(lines_image, (X1, Y1), (X2, Y2), (255,0,0), 10)
            return lines_image

In [37]:
image = cv2.imread('test_image.jpg')

In [38]:
lanelines_image = np.copy(image)

In [39]:
canny_conv = canny_edge(lanelines_image)

In [40]:
cropped_image = reg_of_interest(canny_conv)

`cv2.HoughLinesP(image, rho, theta, threshold, array, minLineLength, maxLineGap)`

Parameter	Description:
    
- `image`	Input binary image
- `rho=2`	Distance resolution of the accumulator in pixels.
- `theta=np.pi/180`	Angular resolution of the accumulator in radians (1-degree step).
- `threshold=100`	Minimum number of votes needed to be considered a line.
- `np.array([])`	Placeholder for optional parameters (not used).
- `minLineLength=40`	Minimum length of a line (shorter segments are discarded).
- `maxLineGap=5`	Maximum gap allowed between points on the same line (to connect broken lines).

In [41]:
# returns detected lines and assign them to lane_lines
lane_lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength= 40, maxLineGap=5)

In [42]:
linelines_image = show_lines(lanelines_image, lane_lines)

In [43]:
cv2.imshow('result', linelines_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Combining with actual image

In [44]:
image = cv2.imread('test_image.jpg')
lane_image = np.copy(image)
canny = canny_edge(lane_image)
cropped_image = reg_of_interest(canny)
lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength= 40, maxLineGap=5)
lines_image = show_lines(lane_image, lines)
combine_image = cv2.addWeighted(lane_image, 0.8, lines_image, 1, 1) # Combine this with lane detection by overlaying the lines onto the original image

In [45]:
cv2.imshow('result', combine_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Above we have learned how to identify lane lines in an image. We took these lines and placed them on a random black image that has the same dimensions as our original image. 
By blending the two, we were able to ultimately place our detected lines back onto our original image.

## Optimizing the detected road marking in images

In [46]:
#Optimization the detected road markings

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [47]:
# this function will collect the line_parameters value (the average of all the slopes) from the average_slope_intercept fucntion and unpack it. We will set y2 to 3/5 * y1 as we want to consider 
# the line up to 3/5 of the y-axis. We know that the equation of a straight line is y = m*x + c, so we can rewrite it. We are going to find x1, y1 and x2, y2 by using the following function.
def make_coordinates(image, line_parameters):
          slope, intercept = line_parameters
          y1 = image.shape[0]
          y2 = int(y1*(3/5))
          x1 = int((y1- intercept)/slope)
          x2 = int((y2 - intercept)/slope)
          return np.array([x1, y1, x2, y2])

In [48]:
# this function averages out the slopes and y-intercepts into a single-line
def average_slope_intercept(image, lines):
          image_width = image.shape[1]
          image_height = image.shape[0]
          
          # left_fit and right_fit are the lists that collected the coordinates of the average value of the lines on the left and right sides.
          left_fit = []
          middle_fit = []
          right_fit = []
          # here, we looped all the lines and reshaped them into a four-dimensional array using line.reshape(4).  
          for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            # fits a first-degree polynomial (a linear function)
            # it fits the polynomial of x and y and returns a vector of coefficients that describes the slope and intercept of a line.
            parameter = np.polyfit((x1, x2), (y1, y2), 1) # fits the line for (x1, y1) and (x2, y2)
            slope = parameter[0]
            intercept = parameter[1]
            # we know that the value of the slope is always negative for the left side of the line, and we wrote a condition to append all the slope values on the left side and right side of the image. Since, origina is at the top-left, x is horizantal and y is vertical axe.
            if x1 < image_width / 5 and slope < 0:
              left_fit.append((slope, intercept))
            elif image_width / 5 <= x1 and x1 <= 1 * image_width / 2 and slope < 0:
              middle_fit.append((slope, intercept))
            else:
              right_fit.append((slope, intercept))
          # then, we averaged out the intercepts (slope and y-intercept) of the left side and right side using np.average
          left_fit_average =np.average(left_fit, axis=0)
          middle_fit_average = np.average(middle_fit, axis=0)
          right_fit_average = np.average(right_fit, axis =0)
          # find the x and y coordinates of the line using the make_coordinates function
          left_line = make_coordinates(image, left_fit_average)
          middle_line = make_coordinates(image, middle_fit_average)
          right_line = make_coordinates(image, right_fit_average)
          
          return np.array([left_line, middle_line, right_line])

In [49]:
def canny_edge(image):
         gray_coversion= cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
         blur_conversion = cv2.GaussianBlur(gray_conversion, (5,5),0)
         canny_conversion = cv2.Canny(blur_conversion, 50,150)
         return canny_conversion

In [50]:
def show_lines(image, lines):
          lanelines_image = np.zeros_like(image)
          if lines is not None:
            for line in lines:
              X1, Y1, X2, Y2 = line.reshape(4)
              cv2.line(lanelines_image, (X1, Y1), (X2, Y2), (255,0,0), 10)
          return lanelines_image

In [51]:
def reg_of_interest(image):
          image_height = image.shape[0]
          # polygons = np.array([[(200, image_height), (1100, image_height), (551, 250)]])
          
          # 4-dimensional polygon
          polygons = np.array([[(0, int(image_height)), (1100, image_height), (550, 250), (0, int(image_height*3/4))]])
          image_mask = np.zeros_like(image)
          cv2.fillPoly(image_mask, polygons, (255, 255, 255))
          masking_image = cv2.bitwise_and(image, image_mask)
          return masking_image

In [52]:
image = cv2.imread('test_image.jpg')

In [53]:
lanelines_image = np.copy(image)

In [54]:
canny_image = canny_edge(lanelines_image)

In [55]:
cropped_image = reg_of_interest(canny_image)

In [56]:
lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength= 40, maxLineGap=5)

In [57]:
averaged_lines = average_slope_intercept(lanelines_image, lines)

In [58]:
line_image = show_lines(lanelines_image, averaged_lines)

In [59]:
combine_image = cv2.addWeighted(lanelines_image, 0.8, line_image, 1, 1)

In [60]:
cv2.imshow('result', combine_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Detecting road markings in video

In [61]:
#Detecting road markings in video

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [62]:
def make_coordinates(image, line_parameters):
          try:
            slope, intercept = line_parameters
          except TypeError:
            slope, intercept = 0.001,0
          #slope, intercept = line_parameters
          y1 = image.shape[0]
          y2 = int(y1*(3/5))
          x1 = int((y1- intercept)/slope)
          x2 = int((y2 - intercept)/slope)
          return np.array([x1, y1, x2, y2])

In [63]:
def average_slope_intercept(image, lines):
          image_width = image.shape[1]
          image_height = image.shape[0]
          
          # left_fit and right_fit are the lists that collected the coordinates of the average value of the lines on the left and right sides.
          left_fit = []
          middle_fit = []
          right_fit = []
          # here, we looped all the lines and reshaped them into a four-dimensional array using line.reshape(4).  
          for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            # fits a first-degree polynomial (a linear function)
            # it fits the polynomial of x and y and returns a vector of coefficients that describes the slope and intercept of a line.
            parameter = np.polyfit((x1, x2), (y1, y2), 1) # fits the line for (x1, y1) and (x2, y2)
            slope = parameter[0]
            intercept = parameter[1]
            # we know that the value of the slope is always negative for the left side of the line, and we wrote a condition to append all the slope values on the left side and right side of the image. Since, origina is at the top-left, x is horizantal and y is vertical axe.
            if slope < 0: # x1 < image_width / 6 and slope < 0:
              left_fit.append((slope, intercept))
            #elif image_width / 6 <= x1 and x1 <= 1 * image_width / 2 and slope < 0:
            #  middle_fit.append((slope, intercept))
            else:
              right_fit.append((slope, intercept))
          # then, we averaged out the intercepts (slope and y-intercept) of the left side and right side using np.average
          left_fit_average =np.average(left_fit, axis=0)
          # middle_fit_average = np.average(middle_fit, axis=0)
          right_fit_average = np.average(right_fit, axis =0)
          # find the x and y coordinates of the line using the make_coordinates function
          left_line = make_coordinates(image, left_fit_average)
          # middle_line = make_coordinates(image, middle_fit_average)
          right_line = make_coordinates(image, right_fit_average)
          
          return np.array([left_line, # middle_line, 
                           right_line])

In [64]:
def canny_edge(image):
         gray_conversion= cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
         blur_conversion = cv2.GaussianBlur(gray_conversion, (5,5),0)
         canny_conversion = cv2.Canny(blur_conversion, 50,150)
         return canny_conversion

In [65]:
def show_lines(image, lines):
          line_image = np.zeros_like(image)
          if lines is not None:
            for line in lines:
              x1, y1, x2, y2 = line.reshape(4)
              cv2.line(line_image, (x1, y1), (x2, y2), (255,0,0), 10)
          return line_image

In [66]:
def reg_of_interest(image):
          image_height = image.shape[0]
          polygons = np.array([[(200, image_height), (1100, image_height), (550, 250)]])
          
          # 4-dimensional polygon
          # polygons = np.array([[(0, int(image_height)), (1100, image_height), (550, 250), (0, int(image_height*3/4))]])
          image_mask = np.zeros_like(image)
          cv2.fillPoly(image_mask, polygons, 255)
          masking_image = cv2.bitwise_and(image,image_mask)
          return masking_image

In [67]:
cap = cv2.VideoCapture("test2.mp4")

In [68]:
# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video file")
    exit()

# Get video properties
frame_width = int(cap.get(3))  # Width
frame_height = int(cap.get(4)) # Height
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second

In [69]:
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # Use 'XVID' for .avi
out = cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))

In [70]:
while(cap.isOpened()):
            ret, frame = cap.read()  # ret: Boolean (True if frame is read correctly)
            
            if not ret:
                print("End of video or cannot fetch frame.")
                break  # Exit loop if frame is empty
            
            # Process the frame
            canny_image = canny_edge(frame)
            cropped_canny = reg_of_interest(canny_image)
            lines = cv2.HoughLinesP(cropped_canny, 2, np.pi/180, 100, np.array([]), minLineLength=40,maxLineGap=5)
            averaged_lines = average_slope_intercept(frame, lines)
            line_image = show_lines(frame, averaged_lines)
            combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
            
            # Write processed frame to output video
            out.write(combo_image)
            
            # Show the result
            cv2.imshow("result", combo_image)
            
            # Quit when 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
cap.release()
out.release()  # Save the video file
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\CavaJ\anaconda3\envs\tf_env\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\CavaJ\anaconda3\envs\tf_env\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


End of video or cannot fetch frame.
